# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [37]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key


In [38]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tiksi,71.6872,128.8694,9.34,67,94,2.35,RU,1722790986
1,1,muisne,0.6000,-80.0333,25.65,81,98,6.23,EC,1722791203
2,2,shar,49.5886,81.0453,23.70,33,0,3.97,KZ,1722791204
3,3,puerto natales,-51.7236,-72.4875,0.25,100,100,3.09,CL,1722790927
4,4,vasteras,59.6162,16.5528,20.48,72,20,2.57,SE,1722791206


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.8,
    cmap= "tab20"
)

# Display the map
map_plot_1


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values

# Drop any rows with null values
city_data_df = pd.DataFrame(city_data_df.loc[
                            (city_data_df["Max Temp"] > 21) &
                            (city_data_df["Max Temp"] < 27) &
                            (city_data_df["Wind Speed"] < 4.5) &
                            (city_data_df["Cloudiness"] == 0)])
city_data_df = city_data_df.dropna(how='any')

# Display sample data
city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,shar,49.5886,81.0453,23.70,33,0,3.97,KZ,1722791204
137,137,arraial do cabo,-22.9661,-42.0278,26.92,61,0,3.60,BR,1722791354
242,242,portland,45.5234,-122.6762,25.55,66,0,0.46,US,1722791445
249,249,havre,49.5833,0.3333,21.30,52,0,3.09,FR,1722791476
350,350,aleksotas,54.8833,23.9000,24.02,77,0,0.51,LT,1722791584
431,431,charikar,35.0136,69.1714,23.48,51,0,1.05,AF,1722791676
447,447,shakawe,-18.3667,21.8500,22.44,8,0,1.97,BW,1722791693
463,463,tughyl,47.7252,84.2063,24.03,42,0,1.68,KZ,1722791713
520,520,beroroha,-21.6667,45.1667,22.39,21,0,2.00,MG,1722791779
559,559,baker city,44.7749,-117.8344,23.84,46,0,0.00,US,1722791609


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = {"City": city_data_df["City"],
            "Country": city_data_df["Country"],
            "Lat": city_data_df["Lat"],
            "Lng": city_data_df["Lng"],
            "Humidity": city_data_df["Humidity"]}
hotel_df = pd.DataFrame(hotel_df)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
2,shar,KZ,49.5886,81.0453,33,
137,arraial do cabo,BR,-22.9661,-42.0278,61,
242,portland,US,45.5234,-122.6762,66,
249,havre,FR,49.5833,0.3333,52,
350,aleksotas,LT,54.8833,23.9000,77,
431,charikar,AF,35.0136,69.1714,51,
447,shakawe,BW,-18.3667,21.8500,8,
463,tughyl,KZ,47.7252,84.2063,42,
520,beroroha,MG,-21.6667,45.1667,21,
559,baker city,US,44.7749,-117.8344,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = "accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, "City"]} - nearest hotel: {hotel_df.loc[index, "Hotel Name"]}")

# Display sample data
hotel_df


Starting hotel search
shar - nearest hotel: Бейсенов
arraial do cabo - nearest hotel: No hotel found
portland - nearest hotel: the Hoxton
havre - nearest hotel: Le Fécamp
aleksotas - nearest hotel: Best Western Santaka
charikar - nearest hotel: صالون عروسی پروان سبز
shakawe - nearest hotel: EdenWood guesthouse
tughyl - nearest hotel: No hotel found
beroroha - nearest hotel: No hotel found
baker city - nearest hotel: Geiser Grand Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
2,shar,KZ,49.5886,81.0453,33,Бейсенов
137,arraial do cabo,BR,-22.9661,-42.0278,61,No hotel found
242,portland,US,45.5234,-122.6762,66,the Hoxton
249,havre,FR,49.5833,0.3333,52,Le Fécamp
350,aleksotas,LT,54.8833,23.9000,77,Best Western Santaka
431,charikar,AF,35.0136,69.1714,51,صالون عروسی پروان سبز
447,shakawe,BW,-18.3667,21.8500,8,EdenWood guesthouse
463,tughyl,KZ,47.7252,84.2063,42,No hotel found
520,beroroha,MG,-21.6667,45.1667,21,No hotel found
559,baker city,US,44.7749,-117.8344,46,Geiser Grand Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [42]:
# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    alpha = 0.8,
    cmap= "tab20",
    hover_cols=["City", "Hotel Name", "Country"]
)

# Display the map
map_plot_2


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)